In [1]:
import tensorflow as tf

In [2]:
wordEmbeddingDimension = 50
vocabularySize=100
labels=10
filterSizes_paragraph = [3]
filterSizes_allPara=3
paragraphLength=10
num_filters_parargaph=15
num_filters_allPara=20
maxParagraph = 5

filterShapeOfAllPara =[filterSizes_allPara,3,1,num_filters_allPara]
fullyConnectedLayerInput = 150


In [3]:
import numpy as np
class DataParser:
    def __init__(self):
        self.data=[]
        self.maxParagraph=5
        self.maxParagraphLength=10
        
    
    def getDataFromfile(self,fname):
        labels = np.floor(np.random.rand(5)*10)
        labels=labels.astype(int)
        paragraphs = [[1,2,3,4,5,6,7,8,9,10],[1,2,3,4,5,6,7,8,9,10]]
        self.data=[(labels,paragraphs)]
        
        
        labels = np.floor(np.random.rand(5)*10)
        labels=labels.astype(int)
        paragraphs = [[1,2,3,4,5,6,7,8,9,10],[1,2,3,4,5,6,7,8,9,10]]
        
        self.data.append((labels,paragraphs))
        
        self.totalPages = len(self.data)
        
    def nextBatch(self):
        
        label = np.zeros(10,dtype=float)
        data = self.data[0]
        dupPara = np.zeros(self.maxParagraphLength,dtype=int) # [0]*self.maxParagraphLength
        para=[]
        for l in data[0]:
            label[l]=1
        
        for d in data[1]:
            para.append(d)
        
#         print(len(para),self.maxParagraph)
        for i in range(len(para),self.maxParagraph):
            para.append(dupPara)
        
        return (label,para)
n = DataParser()    
n.getDataFromfile("some")
print(n.nextBatch())
print(n.totalPages)

(array([ 0.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,  1.]), [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])])
2


In [4]:
wordEmbedding = tf.Variable(tf.random_uniform([vocabularySize, wordEmbeddingDimension], -1.0, 1.0),name="wordEmbedding")

paragraphList = []
for i in range(maxParagraph):
    paragraphList.append(tf.placeholder(tf.int32,[paragraphLength],name="paragraphPlaceholder"+str(i)))

target = tf.placeholder(tf.float32,[labels],name="target")

In [5]:
paragraph=[[1,2,3,4,5,6,7,8,9,10],[1,2,3,4,5,6,7,8,9,10]]


In [6]:
def getParagraphEmbedding(paragraphWords):
    paraEmbedding=tf.nn.embedding_lookup(wordEmbedding,paragraphWords)
    
    return tf.expand_dims(tf.expand_dims(paraEmbedding, -1),0)

In [7]:
# paragraphVector = getParagraphEmbedding(paragraphList[0])

In [8]:
# paragraphVector

In [9]:
def convLayeronParagraph(paragraphVector,filterSizes,num_input_channels,num_filters):
    
    pooled_outputs=[]
    for filter_size in filterSizes:
        shape = [filter_size,wordEmbeddingDimension,1,num_filters]
        
        weights = tf.Variable(tf.truncated_normal(shape, stddev=0.1),name="paragraphConvLayerW_"+str(filter_size))
        bias= tf.Variable(tf.constant(0.1, shape=[num_filters]),name="paragraphConvLayerB_"+str(filter_size))
        conv = tf.nn.conv2d(
                    paragraphVector,
                    weights,
                    strides=[1, 1, wordEmbeddingDimension, 1],
                    padding="SAME",
                    name="conv")
        
        h = tf.nn.relu(tf.nn.bias_add(conv, bias), name="relu")
        pool_length=5
        pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, pool_length, 1, 1],
                    strides=[1, pool_length, 1, 1],
                    padding='SAME',
                    name="pool")
        pooled_outputs.append(pooled)
    return tf.reshape(tf.concat(pooled_outputs,axis=0),[1,-1])
#     filter_size = 

In [10]:
# paragraphCNNOutput = convLayeronParagraph (paragraphVector,filterSizes,1,num_filters_parargaph)

In [11]:
# paragraphCNNOutput

In [12]:
def convLayerCombineParagraph(paragraphVectorList,filterSizes_paragraph,filterShapeOfAllPara,num_filters_parargaph,num_filters_allPara):
    
    paragraphCNNEmbedding=[]
    
    for paragraph in paragraphVectorList:
        paragraphVector = getParagraphEmbedding(paragraph)
        cnnEmbedding = convLayeronParagraph(paragraphVector,filterSizes_paragraph,1,num_filters_parargaph)
        paragraphCNNEmbedding.append(cnnEmbedding)
    
    allParagraph=tf.expand_dims(tf.expand_dims(tf.concat(paragraphCNNEmbedding,axis=0),-1),0)
    
    shape = filterShapeOfAllPara
    
    weights= tf.Variable(tf.truncated_normal(shape, stddev=0.1),name="paragraphConvLayer2W_"+str(filterShapeOfAllPara[0]))
    bias= tf.Variable(tf.constant(0.1, shape=[num_filters_allPara]),name="paragraphConvLayer2B_"+str(filterShapeOfAllPara[0]))
    
    conv = tf.nn.conv2d(
                    allParagraph,
                    weights,
                    strides=[1, 1, 1, 1],
                    padding="SAME",
                    name="conv")
    h = tf.nn.relu(tf.nn.bias_add(conv, bias), name="relu")
    return tf.reshape(allParagraph,[1,-1])
    

In [13]:
convOutput=convLayerCombineParagraph(paragraphList,filterSizes_paragraph,filterShapeOfAllPara,num_filters_parargaph,num_filters_allPara)
# print(convOutput)

In [14]:
def fullyConnectedLayer(convOutput,labels):
    shape = [fullyConnectedLayerInput,labels]
    weights =tf.Variable(tf.truncated_normal(shape, stddev=0.1),name="FC_W")
    bias = tf.Variable(tf.constant(0.1, shape=[labels]),name="FC_Bias")
    layer = tf.nn.sigmoid(tf.matmul(convOutput, weights) + bias)
    return layer

In [15]:
prediction=fullyConnectedLayer(convOutput,labels)


In [16]:
cross_entropy = -tf.reduce_sum(((target*tf.log(prediction + 1e-9)) + ((1-target) * tf.log(1 - prediction + 1e-9)) )  , name='xentropy' ) 
cost = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

In [17]:
session = tf.Session()
session.run(tf.global_variables_initializer())

In [19]:
epoch = 10
for e in range(epoch):
    for itr in range(n.totalPages):
        feed_dict_input={}
        data= n.nextBatch()
        feed_dict_input[target]=data[0]
    #     print(target)
    #     print(data[0])
        for p in range(maxParagraph):
            feed_dict_input[paragraphList[p]]= data[1][p]
        cross_e=session.run(optimizer,feed_dict=feed_dict_input)
        cross_e=session.run(cross_entropy,feed_dict=feed_dict_input)
        print(cross_e)

7.72125
7.61157
7.50285
7.39522
7.28894
7.18374
7.0795
6.97785
6.87802
6.77966
6.68362
6.58883
6.49629
6.40476
6.31409
6.22477
6.13654
6.04996
5.96495
5.88173


In [19]:
target

<tf.Tensor 'Placeholder_5:0' shape=(10,) dtype=float32>

In [20]:
saver = tf.train.Saver()

In [21]:
saver.save(session, 'my_test_model')


 

'my_test_model'

In [22]:
cross_e=session.run(cross_entropy,feed_dict=feed_dict_input)
print(cross_e)
pred=session.run(prediction,feed_dict=feed_dict_input)
print(pred)

5.88173
[[ 0.48465297  0.46795458  0.55421132  0.58471745  0.52908742  0.65127993
   0.76014119  0.74830484  0.34570143  0.63871753]]


In [23]:
session = tf.Session()
# session.run(tf.global_variables_initializer())
new_saver = tf.train.import_meta_graph('my_test_model.meta')
new_saver.restore(session, './my_test_model')


In [21]:
# session.run(tf.global_variables_initializer())
# session.run(tf.all_variables())
# 

Instructions for updating:
Please use tf.global_variables instead.


[array([[-0.34391594, -0.56240058, -0.30187535, ...,  0.15665722,
          0.54260206,  0.89840508],
        [-0.49113297, -0.39764094,  0.8171711 , ..., -0.88577223,
          0.03514171, -0.31196737],
        [-0.37415695, -0.63003182,  0.65004277, ...,  0.76201129,
          0.2338531 ,  0.99601746],
        ..., 
        [ 0.2544632 ,  0.90432787,  0.23903394, ..., -0.38299012,
          0.8958087 ,  0.93840098],
        [-0.10583019, -0.56988811,  0.47546411, ...,  0.50619459,
         -0.00968432, -0.62878537],
        [-0.39321089,  0.80393672,  0.28709936, ..., -0.42323542,
         -0.87922072,  0.49390602]], dtype=float32),
 array([[[[ 0.13817279,  0.19292444,  0.16423476, ...,  0.01886966,
           -0.03780412,  0.00108026]],
 
         [[ 0.12091976,  0.04771323, -0.0103507 , ...,  0.07343178,
           -0.02402071, -0.00173428]],
 
         [[ 0.02179516,  0.12996945,  0.01704216, ...,  0.06564666,
           -0.02433765, -0.12003338]],
 
         ..., 
         [[ 0.0

In [24]:
print(session.run(prediction,feed_dict=feed_dict_input))
session.run(cross_entropy,feed_dict=feed_dict_input)

[[ 0.48465297  0.46795458  0.55421132  0.58471745  0.52908742  0.65127993
   0.76014119  0.74830484  0.34570143  0.63871753]]


5.8817272

In [25]:
all_vars = tf.trainable_variables()
for v in all_vars:
    print(v)
    print(session.run(v))

Tensor("wordEmbedding/read:0", shape=(100, 50), dtype=float32)
[[ 0.15876196  0.03288075 -0.11938202 ..., -0.85356599 -0.25316444
  -0.66817486]
 [-0.80373013  0.84529328 -0.0280884  ...,  0.29843634  0.81987017
  -0.91085243]
 [-0.14685997  0.36740819 -0.79256481 ...,  0.80585134  0.65646827
   0.63290709]
 ..., 
 [-0.94680476  0.28128099 -0.16067362 ..., -0.74531531  0.66929793
   0.36764741]
 [ 0.61621833 -0.2775178   0.13468742 ...,  0.5144043   0.73990011
   0.39613795]
 [-0.405442    0.1760838   0.05926657 ...,  0.93862844  0.50915027
  -0.67611408]]
Tensor("paragraphConvLayerW_3/read:0", shape=(3, 50, 1, 15), dtype=float32)
[[[[ 0.01258376 -0.02009754  0.04771244 ...,  0.03519841 -0.02646085
     0.05196681]]

  [[ 0.03849616 -0.14151998  0.07561936 ..., -0.06784324 -0.00873783
    -0.00734385]]

  [[ 0.11256834  0.04363953 -0.17367938 ..., -0.06777207  0.00330694
    -0.08981721]]

  ..., 
  [[ 0.02007247 -0.03413174 -0.15574868 ...,  0.05175923  0.08288772
    -0.18177009]]

 

In [26]:
print([v.op.name for v in tf.all_variables()])

Instructions for updating:
Please use tf.global_variables instead.
['wordEmbedding', 'paragraphConvLayerW_3', 'paragraphConvLayerB_3', 'paragraphConvLayerW_3_1', 'paragraphConvLayerB_3_1', 'paragraphConvLayerW_3_2', 'paragraphConvLayerB_3_2', 'paragraphConvLayerW_3_3', 'paragraphConvLayerB_3_3', 'paragraphConvLayerW_3_4', 'paragraphConvLayerB_3_4', 'paragraphConvLayer2W_3', 'paragraphConvLayer2B_3', 'FC_W', 'FC_Bias', 'beta1_power', 'beta2_power', 'wordEmbedding/Adam', 'wordEmbedding/Adam_1', 'paragraphConvLayerW_3/Adam', 'paragraphConvLayerW_3/Adam_1', 'paragraphConvLayerB_3/Adam', 'paragraphConvLayerB_3/Adam_1', 'paragraphConvLayerW_3_1/Adam', 'paragraphConvLayerW_3_1/Adam_1', 'paragraphConvLayerB_3_1/Adam', 'paragraphConvLayerB_3_1/Adam_1', 'paragraphConvLayerW_3_2/Adam', 'paragraphConvLayerW_3_2/Adam_1', 'paragraphConvLayerB_3_2/Adam', 'paragraphConvLayerB_3_2/Adam_1', 'paragraphConvLayerW_3_3/Adam', 'paragraphConvLayerW_3_3/Adam_1', 'paragraphConvLayerB_3_3/Adam', 'paragraphConvLa

In [2]:
ckpt = tf.train.get_checkpoint_state('.')

In [3]:
sess = tf.Session()

In [4]:
saver = tf.train.Saver()
saver.restore(sess, ckpt.model_checkpoint_path)

ValueError: No variables to save